# FX AI Analytics — Exploratory Notebook

This notebook demonstrates how to programmatically use functions from `app.py` without Streamlit UI: load data, train models, calculate news and events, get the final signal and visualize price and ATR(14).

Below are the steps: imports, instrument configuration, calculations and charts, and at the end — text recommendations and possible bottlenecks of the current approach.

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import plotly.graph_objects as go

sys.path.append(os.path.abspath("."))

from app import (
    load_price_data,
    add_features,
    add_targets,
    train_models,
    compute_news_sentiment,
    fetch_future_economic_events,
    score_future_events,
    detect_patterns,
    build_price_chart,
    build_atr_chart,
    build_prediction_chart,
    build_classification_chart,
    build_classification_comparison_chart,
    combine_signals,
    get_atr_volatility_info,
    enrich_signals_with_atr,
    get_signals_for_ticker,
)

## 1. Instrument Configuration

Set the ticker, human-readable name, history depth and timeframe. The profile (conservative/aggressive, etc.) can be left as `None`, then default settings from `INSTRUMENT_SETTINGS` inside `app.py` will be used.

In [ ]:
ticker = "EURUSD=X"
instrument_name = "EUR/USD"
years = 5
interval = "1d"
profile = None

config = {
    "ticker": ticker,
    "instrument_name": instrument_name,
    "years": years,
    "interval": interval,
    "profile": profile,
}
config

## 2. Quick Unified Pipeline Call

Using the high-level function `get_signals_for_ticker`, which internally:
- loads price history via `yfinance`;
- builds features, targets and trains models;
- calculates news and future events;
- builds patterns and integral scoring;
- adds the latest ATR(14) values and volatility level to the signal.

This is a convenient entry point for API and monitoring.

In [ ]:
signal_payload = get_signals_for_ticker(
    ticker=ticker,
    instrument_name=instrument_name,
    years=years,
    interval=interval,
    profile=profile,
)
signal_payload

## 3. Detailed Step-by-Step Pipeline

Now let's repeat the key steps separately: data loading, feature engineering, model training and signal combination. This is useful for debugging and experiments.

In [ ]:
df_raw = load_price_data(ticker, years=years, interval=interval)
df_full = add_features(df_raw)
atr_info = get_atr_volatility_info(df_full)

horizon = 7
lower_q = 0.33
upper_q = 0.66

df_model = add_targets(
    df_full.copy(),
    horizon=horizon,
    lower_q=lower_q,
    upper_q=upper_q,
)
model_data = train_models(df_model)
metrics = model_data.get("metrics")
df_full.tail()

## 4. News, Events, Patterns and Final Signal

Calculating news sentiment, future macro-events, price patterns and assembling the final integral signal, including ATR(14) information.

In [ ]:
news_items, news_score = compute_news_sentiment(ticker, instrument_name)
future_events = fetch_future_economic_events(days_ahead=7)
future_events_score = score_future_events(future_events)
patterns = detect_patterns(df_full["Close"])

model_weights = metrics.get("weights") if metrics else None
price_model = "hybrid" if metrics and "hybrid" in metrics else "lstm"
cls_conf_threshold = 0.55

signals = combine_signals(
    df_full,
    model_data,
    news_score,
    patterns,
    future_events_score,
    model_weights,
    price_model,
    classifier_override=None,
    cls_conf_threshold=cls_conf_threshold,
)
signals = enrich_signals_with_atr(signals, atr_info)
signals

## 5. Visualization: Price, ATR(14), Forecasts and Classification

Using the same plotting functions as inside the Streamlit app, but showing them directly in the notebook via Plotly.

In [ ]:
price_fig = build_price_chart(df_full, instrument_name, patterns=patterns)
atr_fig = build_atr_chart(df_full, instrument_name)
pred_fig = build_prediction_chart(df_model, model_data, price_model, model_weights)
class_fig = build_classification_chart(df_model, model_data, classifier_override=None)

price_fig.show()
atr_fig.show()
pred_fig.show()
class_fig.show()

In [ ]:
if metrics:
    comparison_fig = build_classification_comparison_chart(
        df_model,
        model_data,
        model_weights,
        cls_conf_threshold,
    )
    comparison_fig.show()

## 6. Recommendations and Possible Errors/Limitations of the Approach

Below are several practical remarks on the current project:

1. **Limited History**  
   If `yfinance` returns little data (e.g. < 2 years for hourly timeframe), the model may not train well or fail with an error. Solution: increase history depth or switch to a higher timeframe.

2. **Simplistic Features**  
   Current features (RSI, MACD, returns, lags) are basic. For real production, it is worth adding: order book data, volatility surfaces, correlation with other assets (SPX, Gold, Oil), and more complex macro factors.

3. **Simple Backtesting**  
   The metrics shown in the logs (precision/recall, Sharpe) are calculated on a "hold for 1 bar" basis. Real trading requires accounting for spread, commissions, slippage, and execution delay. See `fx_ai_backtest.ipynb` for a more detailed backtest.

4. **Pattern Recognition**  
   The pattern recognition module (`detect_patterns`) is heuristic (local extrema). It may give false positives or miss complex figures. It is worth using specialized libraries or ML for pattern recognition.

5. **News Sentiment**  
   News analysis is based on simple keyword scoring. For better quality, LLM (BERT/GPT) should be used to analyze headlines and summaries, as implemented in the `compute_news_sentiment` stub (if external APIs are connected).

6. **Model Retraining**  
   Markets change. Models need to be retrained regularly (e.g. once a week/month). The current code trains the model from scratch on every call — this is acceptable for a prototype but slow for production. It is better to save/load weights (`joblib`/`pickle`).

7. **Risk Management**  
   The signal gives a direction (-1/0/1). The position size should be determined by the risk block (Risk Manager), which is not fully implemented in this notebook (only basic ATR logic). It is recommended to limit risk per trade (e.g. 1-2% of deposit).

Good luck with your experiments!